In [4]:
!pip install LAC

Looking in indexes: https://pypi.douban.com/simple
  Using cached LAC-2.1.2-py2.py3-none-any.whl
  Using cached https://pypi.doubanio.com/packages/63/5d/381a2a5cc29b1215249688e07a2557f785aea4aa4d5c1ca76272fb35f80f/paddlepaddle-2.1.1-cp37-cp37m-win_amd64.whl (61.5 MB)
  Using cached https://pypi.doubanio.com/packages/ed/1b/72a1821152d07cf1d8b6fce298aeb06a7eb90f4d6d41acec9861e7cc6df0/decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.0.9
    Uninstalling decorator-5.0.9:
      Successfully uninstalled decorator-5.0.9


In [5]:
import pandas as pd
import numpy as np
import re
from LAC import LAC

In [6]:
label2id = {
    '非常不满-服务敏感':0,
    '非常不满-费用敏感':1,
    '非常不满-渠道敏感':2,
    '一般不满':3,
    '比较不满':4
}

C:\softwares\anaconda3\envs\tf2\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
data = pd.read_csv('./data.csv')[['情绪等级标注结果','受理内容']]
print(data.head(1)['受理内容'])

0    19927044318来电反映18026183816号码要求挂失，已和用户解释，该号码实时信...
Name: 受理内容, dtype: object


In [4]:
data = data[['情绪等级标注结果','受理内容']].dropna()
data.情绪等级标注结果.value_counts()

比较不满         15932
非常不满-渠道敏感     6222
一般不满          4111
非常不满-服务敏感     2385
非常不满-费用敏感     1397
Name: 情绪等级标注结果, dtype: int64

In [5]:
data['情绪等级标注结果'] = data['情绪等级标注结果'].apply(lambda x:label2id.get(x))
data

,情绪等级标注结果,受理内容
0,2,19927044318来电反映18026183816号码要求挂失，已和用户解释，该号码实时信...
1,2,【费用争议帐期】0\r\n【争议上网时间】\r\n【争议金额(元)】\r\n用户来电表示其宽...
2,2,【费用争议帐期】2019-08-26\r\n【争议上网时间】2019-08-26 18:48...
3,2,(总经理热线)客户对前单处理结果不满，工单编号：2019082310011783 已归档）\...
4,2,【接入方式】FTTH(光纤到户)\r\n【故障单号】20190826000031010\r\...
...,...,...
30042,4,【客户投诉内容】客户表示现在有201409移动话费促销包送话费_480元_24个月，有效期是...
30043,4,用户来电查询为何名下有1000多款的费用，前台经查询是宽带和手机的5月费到9月费的费用，用户...
30044,4,【订单编码】200190905760998617\r\n【订单环节信息】\r\n【移机或装机...
30045,3,【区/县】 长安\r\n【路名】 大岭山杨屋第三工业区\r\n用户来电反映其手机（18927...


In [6]:
data['受理内容'] = data['受理内容'].apply(lambda x:re.sub('[0-9][0-9]{5,}|[\\n]|[/\\r]|[【]|[】]|[A-Za-z0-9_]{5,}|[,]','',x))
data['受理内容'] = data['受理内容'].apply(lambda x:re.sub('([0-9]{3}[1-9]|[0-9]{2}[1-9][0-9]{1}|[0-9]{1}[1-9][0-9]{2}|[1-9][0-9]{3})-(((0[13578]|1[02])-(0[1-9]|[12][0-9]|3[01]))|((0[469]|11)-(0[1-9]|[12][0-9]|30))|(02-(0[1-9]|[1][0-9]|2[0-8])))|[0-9]{4}[年][0-9]{1,}[月][0-9]{1,}[日]|[0-9]{1,}[:0-9]{1,}[:0-9]{1,}','',x))
# bert allow max_len == 512 包括[CLS][SEP]
data['受理内容'] = data['受理内容'].apply(lambda x:x[:510] if len(x)>510 else x)
data.head()

,情绪等级标注结果,受理内容
0,2,来电反映号码要求挂失，已和用户解释，该号码实时信用额度停机，需预存费用复机后再挂失，或是直接...
1,2,费用争议帐期0争议上网时间争议金额(元)用户来电表示其宽带为何现在又欠费停机了，前台经核实，...
2,2,费用争议帐期争议上网时间 18:48:24争议金额(元)客户投诉内容客户投诉号码 [] 在 ...
3,2,(总经理热线)客户对前单处理结果不满，工单编号： 已归档）前工单反馈内容：（用户来电反映其手...
4,2,接入方式FTTH(光纤到户)故障单号故障现象无法连接上网故障申告时间 19:46:20是否为...


# 数据样本存在严重不均衡的情况，尝试做一下数据增强
## 主要针对非常不满类别下 ：服务敏感、费用敏感，实现思路如下：
1. 从构建的专家词库中选取某类别词
2. 随机选取若干数量区间的词，使得文本长度在100-300之间，对某类别做数据增强，强调某些字对与某个类别的重要性
3. 通过分词构建一个通用词表，保存所有类别中都出现过通用词
4. 对于专家词库数量词数量很少的类别，对原语料该类别做一个分词，把分词结果统计起来，从中剔除通用词表中存在的词，把最终的某类别专用词表更新到对应类别的专家词库中。
5. 对于小样本类别根据构建的专家词库生成新的数据。

In [7]:
data.情绪等级标注结果.value_counts()

4    15932
2     6222
3     4111
0     2385
1     1397
Name: 情绪等级标注结果, dtype: int64

label2id = {'非常不满-服务敏感':0,
        '非常不满-费用敏感':1,
        '非常不满-渠道敏感':2,
        '一般不满':3,
        '比较不满':4}

# 构建通用词表，通过对每个label的corpus分词并两两取交集

## 1.构建每个类别的分词词库

In [8]:
# 对任意两个取交集构成共有词库
# 在小样本类别词库中剔除共有词库中词汇并添加专家词库
data0 = np.asarray(data[data.情绪等级标注结果==0])
data1 = np.asarray(data[data.情绪等级标注结果==1])
data2 = np.asarray(data[data.情绪等级标注结果==2])
data3 = np.asarray(data[data.情绪等级标注结果==3])
data4 = np.asarray(data[data.情绪等级标注结果==4])

## 对不同分类的语料分词

In [9]:
lac = LAC(mode='seg')

In [10]:
corpus0 = ''
for i in range(len(data0)):
    corpus0 += data0[i][1]
corpus0 = set(lac.run(corpus0))

In [11]:
corpus1 = ''
for i in range(len(data1)):
    corpus1 += data1[i][1]
corpus1 = set(lac.run(corpus1))

In [12]:
corpus2 = ''
for i in range(len(data2)):
    corpus2 += data2[i][1]
corpus2 = set(lac.run(corpus2))

In [13]:
corpus3 = ''
for i in range(len(data3)):
    corpus3 += data3[i][1]
corpus3 = set(lac.run(corpus3))

In [14]:
corpus4 = ''
for i in range(len(data4)):
    corpus4 += data4[i][1]
corpus4 = set(lac.run(corpus4))

In [15]:
base_corpus = set(corpus0.intersection(corpus1,corpus2,corpus3,corpus4))

## 新词库去除通用词表中的词

In [16]:
set0 = corpus0.difference(base_corpus)
set1 = corpus1.difference(base_corpus)
set2 = corpus2.difference(base_corpus)

## 合并专家词库

In [17]:
label0 = np.asarray(pd.read_csv('./词库/0.csv')).squeeze()
label1 = np.asarray(pd.read_csv('./词库/1.csv')).squeeze()
label2 = np.asarray(pd.read_csv('./词库/2.csv')).squeeze()
for i in label0:
    set0.add(i)
for j in label1:
    set1.add(j)
for k in label2:
    set2.add(k)
len(set0),len(set1),len(set2)

(7957, 3592, 13435)

In [25]:
lst = []
lst.append(list(set0))
lst.append(list(set1))
lst.append(list(set2))

## 对新的词库序列化存储

In [26]:
import pickle
'''
非常不满-服务敏感':0
非常不满-费用敏感':1
非常不满-渠道敏感':2
'''
with open("非常不满综合词库.pkl",'wb') as f:
    pickle.dump(lst,f)
    f.close()

In [27]:
with open("非常不满综合词库.pkl",'rb') as f:
    print(pickle.load(f))

[['建站', '包侧', '尾数', '广东理工学院', '等等', '9：22', '王胜兵', '向315', '爱民路70号', '邱原芬', '政德街', '庞大', '西瓜', '558.11', '葛先生', '后端', '乌沙社区', '3号户', '南华大街', '越多', '维也纳酒店', '太和营服中心', '偷偷', '石岐区政企营销服务中心', '11:27:00', '集团网络投诉', '超量', '争议强烈', '12:17:38', '2-3格', '2603', '第2年', '润迅固网', '一千元', '2019年8月31日', '10:01:23', '沿江田家炳中学', '积分', '一体化', '16:00:00', '址', '旧版', '17:07:34', '乐享4G套餐增值包', '天数', '签', '先不', '庞晓春', '20g', '龙城', '夏南', '天河', '营业厅厅', '注意', '油', '策划室', '9.30', 'ATM', '9G', '呼出    关机    18:10', '虽', '布', '依赖', '64.78', '情', '白石村塘井小组红二队', '429-99）元', '调配', '西福蓝湾', '过长', '18:00:22', '0692', '铁塔物业', '179', '7P', '工业园', '18:00:00', '12:37', '意见大请', '石排营销服务中心', '时号', '连南县', '@@@JT', '7元', '决现', '2017-7-22日', '68元', '线口', '已有两年', '2019年8月239元', '30日内', '制定', '县路', '霍先生', '文化', ' 坪山营销服务中心', '1410', '125.24元', '后户', '葛优', '不退', '5月31', '福田财富大厦', '乾塘', '269元', '再说', '电联', '160-1-1', '梁智斌', '周围', '凯旋城', '游戏', '机器', '1531.19', '南湾街道办', '十四巷9号', '谢羿干', '现公', '11月14日', '黄柏声    ', '包达量', '发觉', '2019年10月16号', '」

    无效词汇过多，对于小样本造出的语料效果不好，可能起到增加小样本泛化性效果
    可以先使用专家词库进行规则匹配筛选潜在的未筛选出来的样本